In [8]:
import dp

In [13]:
dp.countries_traffic

array(['san marino', 'guernsey uk', 'gibraltar uk', 'jersey uk',
       'andorra', 'liechtenstein', 'brunei', 'taiwan', 'new zealand',
       'united states', 'malta', 'luxembourg', 'isle of man uk',
       'iceland', 'australia', 'guam us', 'italy', 'finland', 'estonia',
       'uruguay', 'portugal', 'malaysia', 'canada', 'poland', 'france',
       'japan', 'norway', 'czech republic', 'bahamas', 'lithuania',
       'spain', 'germany', 'thailand', 'united kingdom', 'switzerland',
       'slovakia', 'belgium', 'antigua and barbuda',
       'trinidad and tobago', 'slovenia', 'austria', 'netherlands',
       'greece', 'cyprus', 'denmark', 'saint kitts and nevis', 'sweden',
       'ireland', 'croatia', 'faroe islands denmark', 'kuwait',
       'costa rica', 'dominican republic', 'qatar', 'south korea',
       'argentina', 'hungary', 'romania', 'bahrain', 'belarus',
       'bulgaria', 'syria', 'latvia', 'libya', 'brazil', 'mexico',
       'georgia', 'israel', 'barbados', 'nauru', 'laos', 's

In [21]:
prefix_length = 3

dp.traffic['country_prefix'] = dp.traffic['Location_normalized'].str[:prefix_length]
dp.forest['country_prefix'] = dp.forest['Country_normalized'].str[:prefix_length]
dp.air_city['country_prefix'] = dp.air_city['Country_normalized'].str[:prefix_length]
dp.air_country['country_prefix'] = dp.air_country['Region_normalized'].str[:prefix_length]
dp.weather['country_prefix'] = dp.weather['country_normalized'].str[:prefix_length]

In [48]:
dp.traffic

,Location,Rate[per 1000],Vehicles,Population,Year,Location_normalized,country_prefix
0,San Marino,"1,606","53,940","33,581",2024,san marino,san
1,Guernsey (UK),"1,389","88,532","63,726",2022,guernsey uk,gue
2,Gibraltar (UK),"1,293","48,641","37,610",2022,gibraltar uk,gib
3,Jersey (UK),"1,234","127,911","103,675",2023,jersey uk,jer
4,Andorra,"1,214","98,183","80,856",2023,andorra,and
...,...,...,...,...,...,...,...
177,Madagascar,9.3,"236,979","25,426,703",2015,madagascar,mad
178,Togo,8.2,"64,118","7,858,289",2016,togo,tog
179,Central African Republic,8.1,"37,475","4,623,182",2014,central african republic,cen
180,Somalia,4.5,"59,457","13,339,491",2014,somalia,som


In [53]:
# traffic-forest

# Find common prefixes (blocking)
TF_common_prefixes = set(dp.traffic['country_prefix']).intersection(set(dp.forest['country_prefix']))
print(f"Common country prefixes are: {TF_common_prefixes}")

# For each common prefix, compare countries
for prefix in TF_common_prefixes:
    subset_TF_a = dp.traffic[dp.traffic['country_prefix'] == prefix]
    subset_TF_b = dp.forest[dp.forest['country_prefix'] == prefix]

    # Cosine similarity matching function for traffic_countries & forest_countries
    def cosine_match(vectorizer, a, b):
        # Transform strings to vectors
        vectors = vectorizer.transform([a, b])
        similarity_score = dp.cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
        return similarity_score >= threshold, similarity_score

    # Initialize TF-IDF vectorizer
    vectorizer = dp.TfidfVectorizer().fit(list(subset_TF_a['Location_normalized'])+list(subset_TF_b['Country_normalized']))

    # Matching process
    matches_tf = []
    for loc in subset_TF_a['Location_normalized'].unique():
        for country in subset_TF_b['Country_normalized'].unique():
            is_match, similarity = cosine_match(vectorizer, loc, country)
            if is_match:
                matches_tf.append({
                    "Prefix": prefix,
                    "Source": "Traffic",
                    "Target": "Forest",
                    "Country_A": loc,
                    "Country_B": country,
                    "Similarity": similarity
                })

    # Convert matches to DataFrame
    tf_df = dp.pd.DataFrame(matches_tf)

    # Display results
    if not tf_df.empty:
        print("Traffic-Forest matches (sorted by similarity):")
        print(tf_df.sort_values('Similarity', ascending=False))
    else:
        print("No Traffic-Forest matches found")


Common country prefixes are: {'tri', 'ban', 'swi', 'fin', 'chi', 'bot', 'mad', 'pol', 'hon', 'slo', 'sol', 'sri', 'new', 'nor', 'tun', 'som', 'sau', 'gam', 'vie', 'ukr', 'dom', 'tha', 'ton', 'tur', 'fra', 'eri', 'sey', 'can', 'leb', 'cha', 'bel', 'ira', 'zam', 'oma', 'pap', 'mic', 'taj', 'cen', 'net', 'ant', 'cos', 'pak', 'gre', 'mon', 'lux', 'bhu', 'isl', 'sou', 'cyp', 'pan', 'bru', 'geo', 'guy', 'ind', 'ger', 'mex', 'fij', 'com', 'bur', 'hun', 'den', 'ven', 'spa', 'ben', 'uru', 'sie', 'rom', 'arm', 'sao', 'and', 'pal', 'kuw', 'lib', 'tan', 'jor', 'cub', 'alg', 'sur', 'uga', 'cam', 'san', 'por', 'ire', 'lat', 'sam', 'jap', 'nam', 'bah', 'gui', 'lit', 'sen', 'nep', 'gha', 'phi', 'jam', 'moz', 'tog', 'alb', 'mya', 'aze', 'bos', 'ice', 'gua', 'uzb', 'rus', 'ken', 'rwa', 'aus', 'egy', 'syr', 'bul', 'lie', 'ser', 'sin', 'isr', 'cze', 'sud', 'cro', 'col', 'qat', 'arg', 'bar', 'lao', 'kyr', 'uni', 'ita', 'mal', 'zim', 'mol', 'swe', 'nig', 'est', 'ecu', 'kir', 'afg', 'kaz', 'per', 'dji', 'far

In [43]:
subset_TF_a

,Location,Rate[per 1000],Vehicles,Population,Year,Location_normalized,country_prefix
117,El Salvador,163,"1,008,080","6,200,800",2016,el salvador,el


In [32]:
threshold = 0.80 

In [41]:
subset_TF_b

,Country Name,Capital,Continent,Area (km²),Population Density (per km²),Population Rank,Forest Area 2010,Forest Area 2011,Forest Area 2012,Forest Area 2013,Forest Area 2014,Forest Area 2015,Forest Area 2016,Forest Area 2017,Forest Area 2018,Forest Area 2019,Forest Area 2020,Country_normalized,City_normalized,country_prefix
171,El Salvador,San Salvador,North America,21041,301.145,112,30.351351,30.13417,29.916988,29.699807,29.482625,29.265444,29.048263,28.831081,28.6139,28.396718,28.179537,el salvador,san salvador,el


In [52]:
# Find common prefixes (blocking)
TF_common_prefixes = set(dp.traffic['country_prefix']).intersection(set(dp.forest['country_prefix']))
print(f"Common country prefixes are: {TF_common_prefixes}")

# Matching process
matches_tf = []

# For each common prefix, compare countries
for prefix in TF_common_prefixes:
    subset_TF_a = dp.traffic[dp.traffic['country_prefix'] == prefix]
    subset_TF_b = dp.forest[dp.forest['country_prefix'] == prefix]

    # Initialize TF-IDF vectorizer for the current prefix
    vectorizer = dp.TfidfVectorizer().fit(list(subset_TF_a['Location_normalized']) + list(subset_TF_b['Country_normalized']))

    # Cosine similarity matching function
    def cosine_match(vectorizer, a, b):
        # Transform strings to vectors
        vectors = vectorizer.transform([a, b])
        similarity_score = dp.cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
        return similarity_score >= threshold, similarity_score

    for loc in subset_TF_a['Location_normalized'].unique():
        for country in subset_TF_b['Country_normalized'].unique():
            is_match, similarity = cosine_match(vectorizer, loc, country)
            if is_match:
                matches_tf.append({
                    "Prefix": prefix,
                    "Source": "Traffic",
                    "Target": "Forest",
                    "Country_A": loc,
                    "Country_B": country,
                    "Similarity": round(similarity, 4)
                })

# Convert matches to DataFrame and display results
tf_df = dp.pd.DataFrame(matches_tf)
if not tf_df.empty:
    print("Traffic-Forest matches (sorted by similarity):")
    print(tf_df.sort_values('Similarity', ascending=False))
else:
    print("No Traffic-Forest matches found")

Common country prefixes are: {'tri', 'ban', 'swi', 'fin', 'chi', 'bot', 'mad', 'pol', 'hon', 'slo', 'sol', 'sri', 'new', 'nor', 'tun', 'som', 'sau', 'gam', 'vie', 'ukr', 'dom', 'tha', 'ton', 'tur', 'fra', 'eri', 'sey', 'can', 'leb', 'cha', 'bel', 'ira', 'zam', 'oma', 'pap', 'mic', 'taj', 'cen', 'net', 'ant', 'cos', 'pak', 'gre', 'mon', 'lux', 'bhu', 'isl', 'sou', 'cyp', 'pan', 'bru', 'geo', 'guy', 'ind', 'ger', 'mex', 'fij', 'com', 'bur', 'hun', 'den', 'ven', 'spa', 'ben', 'uru', 'sie', 'rom', 'arm', 'sao', 'and', 'pal', 'kuw', 'lib', 'tan', 'jor', 'cub', 'alg', 'sur', 'uga', 'cam', 'san', 'por', 'ire', 'lat', 'sam', 'jap', 'nam', 'bah', 'gui', 'lit', 'sen', 'nep', 'gha', 'phi', 'jam', 'moz', 'tog', 'alb', 'mya', 'aze', 'bos', 'ice', 'gua', 'uzb', 'rus', 'ken', 'rwa', 'aus', 'egy', 'syr', 'bul', 'lie', 'ser', 'sin', 'isr', 'cze', 'sud', 'cro', 'col', 'qat', 'arg', 'bar', 'lao', 'kyr', 'uni', 'ita', 'mal', 'zim', 'mol', 'swe', 'nig', 'est', 'ecu', 'kir', 'afg', 'kaz', 'per', 'dji', 'far

In [51]:
# Cosine similarity matching function for traffic_countries & forest_countries
def cosine_match(vectorizer, a, b):
    # Transform strings to vectors
    vectors = vectorizer.transform([a, b])
    similarity_score = dp.cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
    return similarity_score >= threshold, similarity_score

# Initialize TF-IDF vectorizer
vectorizer = dp.TfidfVectorizer().fit(list(subset_TF_a['Location_normalized'])+list(subset_TF_b['Country_normalized']))

# Matching process
matches_tf = []
for loc in subset_TF_a['Location_normalized'].unique():
    for country in subset_TF_b['Country_normalized'].unique():
        is_match, similarity = cosine_match(vectorizer, loc, country)
        if is_match:
            matches_tf.append({
                "Prefix": prefix,
                "Source": "Traffic",
                "Target": "Forest",
                "Country_A": loc,
                "Country_B": country,
                "Similarity": similarity
            })

# Convert matches to DataFrame
tf_df = dp.pd.DataFrame(matches_tf)

# Display results
if not tf_df.empty:
    print("Traffic-Forest matches (sorted by similarity):")
    print(tf_df.sort_values('Similarity', ascending=False))
else:
    print("No Traffic-Forest matches found")

Traffic-Forest matches (sorted by similarity):
  Prefix   Source  Target    Country_A    Country_B  Similarity
0    el   Traffic  Forest  el salvador  el salvador         1.0


In [ ]:
# traffic-forest

# Find common prefixes (blocking)
common_prefixes = set(traffic['country_prefix']).intersection(set(forest['country_prefix']))
print(f"Common country prefixes are: {common_prefixes}")

# For each common prefix, compare countries
for prefix in common_prefixes:
    subset_a = traffic[traffic['country_prefix'] == prefix]
    subset_b = forest[forest['country_prefix'] == prefix]
    
    for country_a in subset_a['Location'].unique():
        for country_b in subset_b['Country Name'].unique():
            score = jw.get_sim_score(str(country_a), str(country_b))
            if score >= threshold:
                matches1.append({
                    "Prefix": prefix,
                    "Country_A": country_a,
                    "Country_B": country_b,
                    "Similarity": round(score, 4)
                })

# Convert to DataFrame and display results
result_df = pd.DataFrame(matches1)
print(result_df)


In [ ]:
# City - Jaro-Winkler
jw = JaroWinkler()
threshold = 0.90
matches2 = []

cities_air_city = air_city['City/Town'].dropna().unique()
cities_air_weather = weather['location_name'].dropna().unique()

In [ ]:
dp.city_forest = forest['City_normalized'].dropna().unique()
dp.city_air_city = air_city['City_normalized'].dropna().unique()
dp.city_air_weather = weather['City_normalized'].dropna().unique()

#Blocking strategy
#Hash Strategy - city to city
#Categorize the cities by their country, and pick an example [ex: China] for comparison


from itertools import product

cities_air_city = air_city['City/Town'].dropna().unique()
cities_air_weather = weather['location_name'].dropna().unique()

# 1. Preprocessing: remove empty values, unify format (lowercase & remove spaces)
air_city['City/Town'] = air_city['City/Town'].str.lower().str.strip()
air_city['Country'] = air_city['Country'].str.lower().str.strip()
weather['location_name'] = weather['location_name'].str.lower().str.strip()
weather['country'] = weather['country'].str.lower().str.strip()

# 2. Filter records whose country is China
cities_air_city_China = air_city[air_city['Country'] == 'China']
cities_air_weather_China = weather[weather['country'] == 'China']

# 3. Finding common cities (Blocking)
common_cities = set(cities_air_city_China['City/Town']).intersection(
    set(cities_air_weather_China['location_name'])
)

print(f"China's common cities are: {common_cities}")

# 4. Make matching combinations in common cities
# air-city-weather
for city_a in dp.city_air_city:
    for city_b in dp.city_air_weather:
        score = jw.get_sim_score(str(city_a), str(city_b))
        if score >= threshold:
            matches2.append({
                "City_a": city_a,
                "City_B": city_b,
                "Similarity": round(score, 4)
            })

# reslt
result_df = pd.DataFrame(matches2)
print(result_df)



In [2]:
import dp

In [3]:
threshold = 0.90

In [6]:
# Cosine similarity matching function for forest - weather

matches_tf = []

def cosine_match(vectorizer, a, b):
    # Transform strings to vectors
    vectors = vectorizer.transform([a, b])
    similarity_score = dp.cosine_similarity(vectors[0:1], vectors[1:2])[0][0]
    return similarity_score >= threshold, similarity_score

# Initialize TF-IDF vectorizer
vectorizer = dp.TfidfVectorizer().fit(list(dp.city_air_city)+list(dp.city_air_weather))

# Matching process

for city_a in dp.city_air_city:
    for city_b in dp.city_air_weather:
        is_match, similarity = cosine_match(vectorizer, city_a, city_b)
        if is_match:
            matches_tf.append({
               "City_a": city_a,
                "City_B": city_b,
                "Similarity": similarity
            })

# Convert matches to DataFrame
tf_df = dp.pd.DataFrame(matches_tf)
tf_df

,City_a,City_B,Similarity
0,tirana,tirana,1.0
1,canberra,canberra,1.0
2,brussels,brussels,1.0
3,dhaka,dhaka,1.0
4,sofia,sofia,1.0
...,...,...,...
61,kampala,kampala,1.0
62,kyiv,kyiv,1.0
63,montevideo,montevideo,1.0
64,san francisco,san francisco,1.0


In [ ]:


for city_a in cities_air_city:
    for city_b in cities_air_weather:
        score = jw.get_sim_score(str(city_a), str(city_b))
        if score >= threshold:
            matches2.append({
                "City_a": city_a,
                "City_B": city_b,
                "Similarity": round(score, 4)
            })

# reslt
result_df = pd.DataFrame(matches2)
print(result_df)

In [ ]:
#Blocking strategy
#Hash Strategy - city to city
#Categorize the cities by their country, and pick an example [ex: China] for comparison


from itertools import product

cities_air_city = air_city['City/Town'].dropna().unique()
cities_air_weather = weather['location_name'].dropna().unique()

# 1. Preprocessing: remove empty values, unify format (lowercase & remove spaces)
air_city['City/Town'] = air_city['City/Town'].str.lower().str.strip()
air_city['Country'] = air_city['Country'].str.lower().str.strip()
weather['location_name'] = weather['location_name'].str.lower().str.strip()
weather['country'] = weather['country'].str.lower().str.strip()

# 2. Filter records whose country is China
cities_air_city_China = air_city[air_city['Country'] == 'China']
cities_air_weather_China = weather[weather['country'] == 'China']

# 3. Finding common cities (Blocking)
common_cities = set(cities_air_city_China['City/Town']).intersection(
    set(cities_air_weather_China['location_name'])
)

print(f"China's common cities are: {common_cities}")

# 4. Make matching combinations in common cities
# air-city-weather
for city_a in cities_air_city:
    for city_b in cities_air_weather:
        score = jw.get_sim_score(str(city_a), str(city_b))
        if score >= threshold:
            matches2.append({
                "City_a": city_a,
                "City_B": city_b,
                "Similarity": round(score, 4)
            })

# reslt
result_df = pd.DataFrame(matches2)
print(result_df)


In [ ]:
#Blocking strategy
#Hash Strategy - city to city
#Categorize the cities by their country, and pick an example [ex: China] for comparison


from itertools import product

cities_air_city = air_city['City/Town'].dropna().unique()
cities_air_weather = weather['location_name'].dropna().unique()

# 1. Preprocessing: remove empty values, unify format (lowercase & remove spaces)
air_city['City/Town'] = air_city['City/Town'].str.lower().str.strip()
air_city['Country'] = air_city['Country'].str.lower().str.strip()
weather['location_name'] = weather['location_name'].str.lower().str.strip()
weather['country'] = weather['country'].str.lower().str.strip()

# 2. Filter records whose country is China
cities_air_city_China = air_city[air_city['Country'] == 'China']
cities_air_weather_China = weather[weather['country'] == 'China']

# 3. Finding common cities (Blocking)
common_cities = set(cities_air_city_China['City/Town']).intersection(
    set(cities_air_weather_China['location_name'])
)

print(f"China's common cities are: {common_cities}")

# 4. Make matching combinations in common cities
# air-city-weather
for city_a in cities_air_city:
    for city_b in cities_air_weather:
        score = jw.get_sim_score(str(city_a), str(city_b))
        if score >= threshold:
            matches2.append({
                "City_a": city_a,
                "City_B": city_b,
                "Similarity": round(score, 4)
            })

# reslt
result_df = pd.DataFrame(matches2)
print(result_df)
